# Algoritmo Genético

    Problema de Mochilas Binário

| Aluno | Matrícula | e-mail |
| :--- | :--- | :--- |
| Gustavo Ribeiro | 2016026329 | gustavo435@ufmg.br |
| Hebert Costa | 2016097439 | hebert15@ufmg.br |


## Descrição

### Descrição Geral

- O algoritmo genético é um método de otimização estocástico baseado na teoria da evolução de Charles Darwin.
- Esta é uma implentação usada para solucionar um problema de mochilas binário;

### O Problema

- Dada um conjunto de dados a respeito de elementos que podem ser incluídos dentro de uma mochila;
- Cada elemento tem um peso e agrega um determinado benefício;
- A mochila tem uma capacidade limitada de carga;
- Nosso objetivo é encontrar a melhor alocação de itens possível de forma a maximizar ao benefício total sem ultrapassar a carga máxima da mochila;

### O Algoritmo

- O algoritmo aborda o problema como se fosse uma população de uma determinada espécia animal;
- A população evolui ao longo de `nGenerations` gerações;
- A cada geração os indivíduos da população atual passam pelos processos de variação e seleção natural;
- O processo de variação inclui a recombinação e a mutação;
- A recombinação corresponde a miscigenação de pais que geram descendentes;
- A mutação consiste em variações aleatórias que podem causar mudanças na constituição dos indivíduos;
- A seleção natural é o processo aonde indivíduos são submetidos a pressão seletiva do meio;
- Apenas indíviduos suficientemente adpatados sobrevivem e tem a chance de gerar novos descendentes;

## Implementação

##### Descrição

- A implementação a seguir contém a definição da classe `Genetic` e as seções de definição de parâmetros e de teste;
- A interface de `Genetic` expõe um único método chamado `evolve`;
- A chamada de `evolve` evoca a "evolução" da espécie desde a sua população inicial até a última geração;
- Internamente a classe possui um método principal chamado `getNewGeneration`;
- Uma nova geração é produzida pela submissão da população atual aos processos de variação e seleção;
- A seleção consiste na chamada subsequente das funções `getRecombined` e `getMutants`;
- `getRecombined` produz novos indivíduos pela mistura de características dos integrantes da geração atual;
- Para isso `getRecombined` utiliza a função `crossover` a qual pode alterar ou não os indivíduos de acordo com uma probabilidade parametrizada;
- `getMutants` pode produz variações nos indivíduos de acordo com uma probabilidade parametrizada;
- A variação de `getMutants` é feita via operação de _Bit Flip_;
- A Seleção é feita pelos métodos `getParents` que seleciona os melhores indivíduos para reprodução;
- A função `getSurvivals` seleciona dentre os indivíduos conhecidos na geração atual quais sobreviverão produzindo a próxima geração;


##### Imports

In [2]:
import numpy as np
import pandas as pd

from typing import Callable

### Algoritmo Genético

#### Utils

In [2]:
def normalize(arr: np.array) -> np.array:
    '''
        TODO: 2022-02-22 - ADD Description
    '''
    
    diff = np.max(arr) - arr
    delta = np.max(arr) - np.min(arr)
    prob = 1 - (diff / delta)
    return prob / np.sum(prob)

def roulette(members: np.array, dbg = False) -> int:
    '''
        Returns the index of a random selected member through turning the wheel of a roulette.
        - The greater the value of a member the bigger the probability of it being selected;
        - A random number is generated simulating a point in the wheel of a roulette;
        - The first member whose accumulated probability range fits the selected point is the winner;

        PARAMS
        - members: Should be an array of numbers
    '''

    m1 = members[0]
    equals, = np.where(members == m1)
    
    areAllEqual = len(equals) == len(members)
    if areAllEqual:
        shares = None
    else:
        shares = normalize(members)

    idx, = np.where(members == np.random.choice(a=members, p=shares))
    return idx[0]

#### Classe Genetic

In [3]:
class Genetic:

    def __init__(self,
        initPopulation: np.array, fitFunc: Callable[[np.array], float],
        crossoverProb: float, mutationProb: float,
        nOffspring: int, nElite: int, nGenerations: int,
    ):
        self.nPopulation, self.nGenes = initPopulation.shape
        self.population = initPopulation
        
        self.nElite = nElite
        self.nOffspring = nOffspring
        self.nGenerations = nGenerations
        
        self.fitFunc = fitFunc
        self.mutationProb = mutationProb
        self.crossoverProb = crossoverProb
    
    def evolve(self) -> tuple:
        '''
            TODO: 2022-02-21 - Rename
            TODO: 2022-02-21 - ADD Description
        '''

        evolution = np.zeros( (self.nGenerations, ) )
        bestFit = -np.inf
        bestGeneration = np.zeros( (self.nPopulation) )

        for i in range(self.nGenerations):
            self.population = self.__getNewGeneration(self.population)
            fits = self.__getFits(generation=self.population, sort=True)
            best = fits[0]
            if best > bestFit:
                bestGeneration = self.population
            evolution[i] = best

        return bestGeneration, evolution

    def __getNewGeneration(self, generation: np.array) -> np.array:
        '''
            TODO: 2022-02-22 - ADD Description
        '''

        # Variation
        mutants = self.__getRecombined(generation)
        mutants = self.__getMutants(mutants)

        # Natural selection
        parents = self.__getParents(mutants)
        survivals = self.__getSurvivals(parents)
        return survivals

    def __getRecombined(self, generation: np.array) -> np.array:
        '''
            TODO: 2022-02-22 - ADD Description
        '''
        
        mutants = np.copy(generation)

        for i in range(1, len(mutants), 2):
            child1, child2 = self.__crossover(generation[i - 1], generation[i])
            mutants[i - 1] = child1
            mutants[i] = child2
        
        return mutants

    def __getMutants(self, generation: np.array) -> np.array:
        '''
            TODO: 2022-02-22 - ADD Description
        '''

        mutants = np.copy(generation)
        for i in range(len(mutants)):
            if np.random.rand() > self.mutationProb:
                mutants[i] = np.logical_not(mutants[i]) # Bit flip

        return mutants

    def __getParents(self, generation: np.array) -> np.array:
        '''
            TODO: 2022-02-21 - ADD Description
        '''

        population = np.concatenate( (generation, self.population) )
        fits = self.__getFits(population)
        
        nParents = self.nOffspring
        parents = np.zeros( (nParents, self.nGenes) )

        # Assure best parents are always preserved
        elite, population, fits = self.__getBestNIndividuals(n=self.nElite, generation=population, fits=fits)
        parents[:self.nElite, :] = elite

        # Select other random parents
        selected, _, _ = self.__getBestNIndividuals(n=nParents - self.nElite, generation=population, fits=fits, testFunc=roulette)
        parents[self.nElite:, :] = selected

        return parents

    def __getSurvivals(self, generation: np.array) -> np.array:
        '''
            TODO: 2022-02-22 - ADD Description
            TODO: 2022-02-22 - Finish implementation
        '''
        return generation

        # if self.nOffspring == self.nPopulation:
        #     return generation # Replace the whole generation

        # survivals = np.zeros( (self.nPopulation, self.nGenes) )
        
        # # Replace with best new individuals
        # newFits = self.__getFits(generation=generation)
        # offspring, _, _ = self.__getBestNIndividuals(n=self.nOffspring, generation=generation, fits=newFits)
        # survivals[:self.nOffspring, :] = offspring

        # # Keep some of the best current individuals
        # currentFits = self.__getFits(generation=self.population)
        # nRemaining = self.nPopulation - self.nOffspring
        # elders, _, _ = self.__getBestNIndividuals(n=nRemaining, generation=self.population, fits=currentFits)
        # survivals[self.nOffspring:, :] = elders

        # return survivals

    def __crossover(self, parent1: np.array, parent2: np.array) -> tuple:
        '''
            TODO: 2022-02-22 - ADD Description
        '''

        willRecombine = np.random.rand() > self.crossoverProb
        if not willRecombine:
            return parent1, parent2

        mid = int(np.floor(self.nGenes / 2))
        
        child1 = np.concatenate( (parent1[:mid], parent2[mid:]) )
        child2 = np.concatenate( (parent2[:mid], parent1[mid:]) )
        return child1, child2

    def __getFits(self, generation: np.array, sort = False) -> np.array:
        '''
            TODO: 2022-02-22 - ADD Description
        '''

        fits = np.apply_along_axis(self.fitFunc, arr=generation, axis=1)
        if sort:
            fits = -np.sort(-fits) # Sort Descending
        return fits

    def __getBestNIndividuals(self, n: int, generation: np.array, fits: np.array, testFunc: Callable[[np.array], bool] = np.argmax) -> tuple:
        '''
            TODO: 2022-02-22 - ADD Description
        '''

        _fits = np.copy(fits)
        _generation = np.copy(generation)

        best = np.zeros( (n, self.nGenes) )
        for i in range(n):
            idx = testFunc(_fits)
            best[i, :] = _generation[idx]
            _fits = np.delete(_fits, idx, axis=0)
            _generation = np.delete(_generation, idx, axis=0)
        
        return best, _generation, _fits

### Configuração

In [4]:
capacity = 35
weights = np.array([10, 18, 12, 14, 13, 11, 8, 6])
values = np.array([5, 8, 7, 6, 9, 5, 4, 3])

nElite = 2
nGenes = len(weights)
nOffspring = 8
nPopulation = 10
nGenerations = 3000

mutationProb = .05
crossoverProb = .6

initPopulation = np.random.randint(low=0, high=2, size=(nPopulation, nGenes))

def fitnessFunction(individual: np.array) -> float:
    '''
        TODO: 2022-02-21 - ADD Description
    '''

    benefit = np.sum(individual * values)
    totalWeight = np.sum(individual * weights)
    
    if totalWeight <= capacity: # Solution is OK
        return benefit

    # Solution is heavier then maximum capacity
    rho = np.max(values / weights) # Penalization factor
    penalties = [(w - capacity) for w in weights] # Penalties for each item
    penalty = rho * np.fabs(sum(penalties))

    return benefit - penalty

#### Teste da função de Aptidão

In [5]:

# Test the case on which all items are included
weightsSum = np.sum(weights)
valuesSum = np.sum(values)
fracs = values / weights
rho  = np.max(fracs)
penalties = [(w - capacity) for w in weights]
penalty = rho * np.fabs(sum(penalties))
allItems = valuesSum - penalty

# print('-------- ALL ITEMS CASE --------')

# print(f'weightsSum = {weightsSum}')
# print(f'valuesSum = {valuesSum}')
# print(f'fracs = {fracs}')
# print(f'rho = {rho}')
# print(f'penalties = {penalties}')
# print(f'penalty = {penalty}')
# print(f'allItems = {allItems}')

# print('--------------------------------')

tests = [
    ([1, 0, 0, 0, 0, 0, 0, 0], 5),
    ([1, 0, 0, 0, 0, 0, 0, 1], 8),
    ([0, 0, 0, 0, 0, 0, 0, 0], 0),
    ([1, 1, 1, 1, 1, 1, 1, 1], allItems),
]

for testIn, testOut in tests:
    fit = fitnessFunction(testIn)
    print(f'{fit} should be: {testOut}')

5 should be: 5
8 should be: 8
0 should be: 0
-83.15384615384616 should be: -83.15384615384616


### Teste

In [6]:
def getGenerationFitness(generation: np.array) -> np.array:
    return np.apply_along_axis(fitnessFunction, arr=generation, axis=1)

def getBestFitness(generation: np.array) -> np.array:
    return np.max(getGenerationFitness(generation))

print(f'initFit: {getBestFitness(initPopulation)} | {getGenerationFitness(initPopulation)}')

genetic = Genetic(
    initPopulation=initPopulation,
    fitFunc=fitnessFunction,
    nElite=nElite,
    nOffspring=nOffspring,
    nGenerations=nGenerations,
    crossoverProb=crossoverProb,
    mutationProb=mutationProb,
)

bestGeneration, evolution = genetic.evolve()

print(f'>> Results:')
print(f'BestFit: {getBestFitness(bestGeneration)}')
print(f'Evolution: {evolution}')
print(f'BestGeneration: {bestGeneration}')


initFit: 11.0 | [ -99.15384615 -111.15384615 -108.15384615 -103.15384615 -107.15384615
 -104.15384615  -93.15384615   11.          -88.15384615  -96.15384615]
>> Results:
BestFit: 19.0
Evolution: [16. 16. 16. ... 19. 19. 19.]
BestGeneration: [[1. 0. 0. 0. 1. 1. 0. 0.]
 [1. 0. 0. 0. 1. 1. 0. 0.]
 [1. 0. 0. 0. 1. 1. 0. 0.]
 [1. 0. 0. 0. 1. 1. 0. 0.]
 [1. 0. 0. 0. 1. 1. 0. 0.]
 [1. 0. 0. 0. 1. 1. 0. 0.]
 [1. 0. 0. 0. 1. 1. 0. 0.]
 [1. 0. 0. 0. 1. 1. 0. 0.]]
